In [3]:
import os
import re
import matplotlib.pyplot as plt
import operator
from datetime import datetime
import pandas as pd
from collections import Counter, defaultdict
from itertools import chain

def getDayValue(s):
    dayValue = s[11:12]
    dayValue = ' '.join(dayValue)
    dayValue = re.sub('일','',dayValue)
    date = datetime.strptime(str(dayValue), '%Y%m%d').strftime('%Y-%m-%d')
    return date

def getRateValue(s):
    rateValue = s[13:14]
    rateValue = ' '.join(rateValue)
    return float(rateValue)

def getBuyMoneyValue(s):
    moneyValue = s[10:11]
    moneyValue = ' '.join(moneyValue)
    moneyValue = re.sub('원','',moneyValue)
    return int(moneyValue)

def getSellMoneyValue(s):
    moneyValue = s[12:13]
    moneyValue = ' '.join(moneyValue)
    moneyValue = re.sub('원','',moneyValue)
    return int(moneyValue)

def getMoneyValue(s):
    moneyValue = s[14:15] 
    moneyValue = ' '.join(moneyValue)    
    return int(moneyValue)

def getCompanyId(s):
    comID = s[5:6] 
    comID = ' '.join(comID)   
    return comID

def getBuyDayValue(s):
    buyDayValue = s[9:10] 
    buyDayValue = ' '.join(buyDayValue) 
    buyDayValue = re.sub('일','',buyDayValue)
    #buyDayValue = re.sub('[*]','',buyDayValue)
    buyDayValue = datetime.strptime(str(buyDayValue), '%Y%m%d').strftime('%Y-%m-%d')
    return buyDayValue

def checkWhitespaceInCompany(s):
    check_s = s.split(' ')
    if len(check_s) == 15 :
        return check_s    
    elif len(check_s) == 16 :
        del(check_s[0])
        return check_s
    elif len(check_s)  == 17:
        del(check_s[0])
        del(check_s[1])
        return check_s
    else :
        return check_s

def getKospiKosdaq(file1, file2):
    kospi = []
    kosdaq = [] 
    while True:
        line = file1.readline()
        if not line: 
            break
        line = line[1:7]
        kospi.append(line)
    while True:
        line = file2.readline()
        if not line: 
            break
        line = line[1:7]
        kosdaq.append(line)
    return kospi, kosdaq

def getKospiKosdaqDayRate(file1, file2):
    kospi_rate = {}
    kosdaq_rate = {}
    for line in file1.readlines():
        check_list = line.split("\t")
        p = re.compile("[0-9]*.[-][0-9]*[-][0-9]") #코스피코스닥 지수 파일에서 <날짜 지수> 라인 찾는 정규표현식
        result = p.search(check_list[0]) #정규표현에 맞는 라인 찾기
        if result:
            day = check_list[0]
            rate = check_list[1]
            kospi_rate[day] = rate
    for line in file2.readlines():
        check_list = line.split("\t")
        p = re.compile("[0-9]*.[-][0-9]*[-][0-9]") #코스피코스닥 지수 파일에서 <날짜 지수> 라인 찾는 정규표현식
        result = p.search(check_list[0]) #정규표현에 맞는 라인 찾기
        if result:
            day = check_list[0]
            rate = check_list[1]
            kosdaq_rate[day] = rate
    return kospi_rate, kosdaq_rate

def getKosRate(day, kos_rate, tf):
    if tf :
        if day in kospi_rate.keys():
            return float(kospi_rate[day])
        else :
            return 0
    else :
        if day in kosdaq_rate.keys():
            return float(kosdaq_rate[day])
        else :
            return 0
        
def checkKospiKosdaq(id, kospi, kosdaq):
    if id in kospi:
        return True
    if id in kosdaq:
        return False

    
def getEngineProfitrate(s):
    rateValue = s.split(" ")
    rateValue = rateValue[3]
    rateValue = re.sub('이익실현','',rateValue)
    return float(rateValue)

def getEngineLossProfitrate(s):
    rateValue = s.split(" ")
    rateValue = rateValue[4]
    rateValue = re.sub('손절','',rateValue)
    return float(rateValue)

if __name__ == '__main__':

    os.chdir('C://Users//송유정//Desktop//Fundsim실험//last_funsim')
    directory = os.listdir() 
    color_list =['r--','bs:','g^-','y--','--k']   

    for file in directory:
        day_list= []
        min_day_list= []
        max_day_list= []
        base_money = 0
        k = 0
        open_file = open(file, 'r')
        if file == "desktop.ini":
            continue
        print(file)
        # 그래프 그리는 변수들
        x_label = []
        y_label = []
        # 그래프 그리는 변수들
        a = 0
        # 날짜별 이윤 계산을 위한 딕셔너리 변수들
        day_profit = {}
        need_money = {} # 날짜별 필요 금액
        sell_m = 0 # 매수 총액
        buy_m = 0 # 매도 총액
        max_need_money = []
        for line in open_file.readlines():
            check_list = line.split("\n")
            p = re.compile("[*][0-9]*.") #펀드심 파일에서 종목코드 종목 매수일 .. 매도일 등락율 투자금 라인 찾는 정규표현식
            result = p.search(check_list[0]) #정규표현에 맞는 라인 찾기
            check_list[0] = checkWhitespaceInCompany(check_list[0]) 
            #print(check_list[0])
            if result:# 정규표현식에 해당하는 line만 검사
                money = getMoneyValue(check_list[0]) # 그 라인의 투자금 get
                buy_money = getBuyMoneyValue(check_list[0]) # 매수 가격(투자금과 나눠서 주 수를 얻기 위함)
                rate = getRateValue(check_list[0])   # 그 라인의 등락률 get 
                profit = (rate/100) * money # 그 날의 이익 계산 (등락률 x 투자금) 
                stock_num = int(money / buy_money) # 투자금과 나눠서 주 수 get
                day = getDayValue(check_list[0]) # 그 라인의 매도일 해당 날짜 get(yyyy-mm-dd 형식)  - 판날
                buyday = getBuyDayValue(check_list[0]) # 그 라인의 매수일 get - 산날                
                companyid = getCompanyId(check_list[0])
                if day in day_profit :
                    day_profit[day] = day_profit.get(day) + profit # 딕셔너리에 이미 키와 profit 있으면 그 키에 다른 종목으로 얻은 profit 더함    
                else :
                    day_profit[day] = profit # 딕셔너리에 처음 들어간 키면 profit 그냥 넣기                    
                buy_m = buy_m + money # 그 라인의 매수액 누적
                sell_m = stock_num * getSellMoneyValue(check_list[0]) # 그 라인의 총 매도액
                #print(buy_m)
                if buyday in need_money: 
                    need_money[buyday] = need_money.get(buyday) + sell_m
                else :
                    need_money[buyday] = buy_m  #매수 누적액 기본 넣기                     
                if day in need_money: #매도가 일어난 날 매도액을 빼기 위함
                    need_money[day] = need_money.get(day) - sell_m
                else :
                    need_money[day] = buy_m - sell_m  #매수액 - 매도액
        day_list = sorted(day_profit.items(), key=operator.itemgetter(0)) # 펀심 결과 수익들을 날짜별로 정렬해서 day_list에 담아 그래프 그릴 준비 
        max_need_money = sorted(need_money.items(), key=operator.itemgetter(1))
        max_need_money = max_need_money[len(max_need_money)-1] # 이 값을 기반으로 수익률 그래프 그리면 됨
        #print(max_need_money)
        base_money = max_need_money[1] # 초기 자본금

        #수익률 구하는 code
        #print(base_money)   
        new_file = open("[result]" + file, 'w')   
        #new_file.write(base_money) //투자금 출력을 위한 line
        for i in range(len(day_profit)):            
            profit_rate = ((day_list[i][1])/base_money)*100 #( (원래돈 + 번돈) - 원래돈 ) / 원래 돈
            #print(day_list[i][0])
            #print(day_list[i][1])
            x_label.append(day_list[i][0]) 
            y_label.append(profit_rate + a)
            k= profit_rate + a
            #print(profit_rate)
            str_line = day_list[i][0] + " %f \n" %k
            new_file.write(str_line)
            a += profit_rate
        print('file end')
        open_file.close()  
        new_file.close()
       # plt.figure(figsize=(100,10))
       # plt.plot(x_label, y_label, color_list[0], label="engine profit")
       # plt.plot(kospi_x_label, kospi_y_label , color_list[1], label="kospi profit")
       # plt.plot(kosdaq_x_label, kosdaq_y_label, color_list[2], label="kosdaq profit")
        #plt.legend()
       # plt.xlabel("Date")
       # plt.xticks(rotation=90)
        #plt.ylabel("Profit")
        #plt.grid(True)     
        #plt.show()

(250)0201011-0.000-0.000-0.990-예측치0.000이상 5억-1조 0.98매수 1.22이익실현 0.90손절 11일보유.txt
file end
(315)0000428-0.000-0.000-0.990-예측치0.000이상 5억-1조 0.94매수 1.24이익실현 0.82손절 28일보유.txt
file end
(715)000017-0.000-0.000-0.990-예측치0.000이상 5억-1조 0.94매수 1.24이익실현 0.88손절 7일보유.txt
file end
(715-pca)000700-0.000-0.000-0.990-예측치0.000이상 5억-1조 0.94매수 1.10이익실현 0.90손절 0일보유.txt
file end
